# RFSoC-PYNQ Multi-Tile Sync Acquisition Design
---

This overlay requires the ZCU208, CLK104 and XM655 boards configured as described in the ["Common Board Setup"](./CommonBoardWiring.ipynb) page.  Please ensure your setup matches that configuration before proceeding.
<p align="center">
  <img src="./images/directConnections_DCblocks.JPG" width="60%" />
</p>
This overlay provides one DAC player memory that is broadcast to multiple RF DAC tiles. DACs in use are Tiles 228 and 229. Tile 230, while enabled, only shares the reference 4GHz to Tiles 228 and 229.  ADC tiles 224 and 225 capture to internal memories.  ADC tile 225 channel 1 stores to the PL DRAM via DMA.  A simplified block diagram is presented below.
<p align="center">
  <img src="./images/daqOverlay_simplifiedBlockDiagram.png" width="65%" />
</p>
<div class="alert alert-block alert-danger">Because of the clocking strategy used in this overlay it is best run after first power-cycling your board.</div>

In [ ]:
from rfsoc_mts import mtsOverlay
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ol = mtsOverlay('mts.bit')

# Generate Waveforms for Loopback
 This overlay uses a single waveform memory that is broadcast to multiple DAC tiles.
### Prepare the DAC Memory
The cell below generates examples waveforms and has customizable paramters.  For example, one can adjust the center frequency, Fc, and re-generate a sinewave.

In [ ]:
from scipy.signal import chirp
from scipy.signal import sawtooth
from scipy.signal import gausspulse

DAC_SR = 4.0E9  # Sample rate of DACs and ADCs is 4.0 GHz
ADC_SR = 4.0E9
Fc = 250.0E6 # Set center frequency of waveform to 250.0 MHz
Fe = 800.0E6 # maximum frequency of chirp at end of record
DAC_Amplitude = 16383.0 # 14bit DAC +16383/-16384
X_axis = (1/DAC_SR) * np.arange(0,ol.dac_player.shape[0])
print (ol.dac_player.shape[0])

In [ ]:
# generate some basic waveforms
DAC_sinewave = DAC_Amplitude * np.sin(2*np.pi*Fc*X_axis)
DAC_sawtooth = DAC_Amplitude * sawtooth(2 * np.pi * Fc * X_axis)
DAC_chirp = DAC_Amplitude * chirp(np.arange(0, ol.dac_player.shape[0])/DAC_SR,
                                  f0=Fc, f1=Fe, t1=(ol.dac_player.shape[0]/DAC_SR), 
                                  phi=0.0, method='linear')
gauss_delay = 4096
DAC_gauss = DAC_Amplitude * gausspulse (np.arange(-gauss_delay, ol.dac_player.shape[0]-gauss_delay)/DAC_SR, fc = Fc,bw = 0.02)

In [ ]:
#ol.dac_player[:] = np.int16(DAC_sinewave)
#ol.dac_player[:] = np.int16(DAC_sawtooth)
#ol.dac_player[:] = np.int16(DAC_chirp)
ol.dac_player[:] = np.int16(DAC_gauss)

---

## Capture DAC Waveform to Internal Memory
The DAC waveform is saved to an internal memory for verification purposes.

In [ ]:
ol.trigger_capture()
pNumSamples =1024*32
plt.title('DAC Waveform Replay ')
plt.xlabel('sample')         
plt.ylabel('Amplitude')
plt.plot(ol.dac_capture[0:pNumSamples])
plt.show()
# Note that the waveform is NOT aligned to sample zero of the DACRAM as it is free-running

---

# Enable Multi-Tile Synchronization - MTS
With Multi-Tile Synchronization enabled the DAC output and ADC samples are aligned.  This feature of the RFSOC allows coherency across DAC and ADC tiles.  The RF tiles are reset and their internal FIFOs reset via "init_tile_sync" while also testing if the synthesizer clocks are ready.  Once initialized, the "sync_tiles" function begins the synchronization procedure of the tiles. The results are analyzed below.

In [ ]:
ol.init_tile_sync()
ol.verify_clock_tree()

In [ ]:
ol.sync_tiles()

---

# Deep Capture to PL DDR4 Memory
The ZCU208 has an additional DRAM exclusively for the PL.  Specifically, this overlay stores ADC Tile 225 - channel 1 samples. Our overlay limits a DMA descriptor to a maximum length of 24bit or 16MB.

In [ ]:
from pynq import allocate
numSamplesPerDMA = (8 << 20)-1 # 8 MSample = 16MB
dbuf = allocate(numSamplesPerDMA, dtype=np.int16, target=ol.ddr4_0)
assert (dbuf.physical_address == ol.ddr4_0.base_address), "Buffer was not allocated to the expected PL-DRAM!"

In [ ]:
ol.dram_capture(dbuf)

## Plot Captured ADC Samples from PL DRAM
The DMA interface captures 2MB worth of sample data and saved it into PL DRAM.  Each sample is 2 bytes each (int16) and we have allocated one mega-sample.

In [ ]:
pStart = 512 * (1<<10)
pNumSamples = 1024*256
plt.plot(ol.ADCdeepcapture[pStart:(pStart+pNumSamples)])
plt.show()

In [ ]:
dbuf.freebuffer() # reclaim memory

## Verify Waveform Properties Match Expectations

If the DAC waveform was a sinewave, verify the captured ADC waveform is sinusoidal.  If a chirp, verify linear frequency ramping of the captured signal or plot the magnitude of the FFT.  Return to earlier cells and try different waveform generation functions and observe the captured ADC samples to the deep memory.  The DMA interface is limited to a maximum of 16MB per descriptor, so it is not feasible to attempt writing the entire DRAM.  One can modify the cells above to target different regions of the DRAM and deposit more sample sets for analysis.

In [ ]:
# Make sure our shared folder is on sys.path (do this once per notebook)
import sys, os
libdir = os.path.expanduser('/home/xilinx/jupyter_notebooks/lib')
if libdir not in sys.path:
    sys.path.insert(0, libdir)

from rfsoc_mts_tdms_export import write_pulses_to_tdms, jupyter_download_link

In [ ]:
# Replace these with your actual variables from the RFSoC-MTS flow:
# dac_wave   -> 1-D array/buffer of your DAC waveform (int16 recommended)
# adc_deep   -> 1-D array/buffer with the complete deep capture from PL-DDR
# fs_dac_hz  -> DAC sample rate (Hz)
# fs_adc_hz  -> ADC sample rate (Hz)

pStart = 512 * (1<<10)
pNumSamples = 1024*1024

tdms_path = write_pulses_to_tdms(
    out_wave=np.int16(DAC_gauss),         # or None to omit channel 'out'
    in_capture=ol.ADCdeepcapture[pStart:(pStart+pNumSamples)],       # required
    fs_out_hz=ADC_SR,
    fs_in_hz=DAC_SR,
    filename="pulses_capture.tdms",
    out_dtype="int16",
    in_dtype="int16",
    group_name="p",
    out_channel="out",
    in_channel="in",
    # properties={"board":"ZCU208","notes":"Gaussian pulses once per buffer"},
)


In [ ]:
# Show a save dialog in the browser
jupyter_download_link(tdms_path)